In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from sklearn.preprocessing import minmax_scale

import os
from pathlib import Path
import re

import tensorflow as tf

In [2]:
train_dir = 'src\images'

In [3]:
train_path = Path(train_dir)

In [4]:
files = list (train_path.glob('*.png'))
files
names = [os.path.split(x)[1] for x in list(train_path.glob('*.png'))]

image_df= pd.concat ([pd.Series(names, name = 'Name'), pd.Series(files, name ='Filepath').astype(str) ], axis= 1)

image_df['Name'] = image_df ['Name'].apply( lambda x: re.sub (r'\.\w+$','',x))

In [5]:
df = pd.read_csv ('src\pokemon.csv')
df

,Name,Type1,Type2
0,bulbasaur,Grass,Poison
1,ivysaur,Grass,Poison
2,venusaur,Grass,Poison
3,charmander,Fire,NaN
4,charmeleon,Fire,NaN
...,...,...,...
804,stakataka,Rock,Steel
805,blacephalon,Fire,Ghost
806,zeraora,Electric,NaN
807,meltan,Steel,NaN


In [6]:
train_df = image_df.merge(df, on= 'Name')
train_df

train_df = train_df.drop('Type2', axis=1)


In [7]:
train_gen = tf.keras.preprocessing.image.ImageDataGenerator(
    validation_split =0.2,
    rescale = 1./255

)

In [8]:
train_data = train_gen.flow_from_dataframe(
    train_df,
    x_col= 'Filepath',
    y_col= 'Type1',
    target_size= (120,120),
    color_node = 'rgba',
    class_node ='sparse',
    batch_size = 32,
    shuffle= True,
    seed= 1,
    subset = 'training'
)

val_data = train_gen.flow_from_dataframe(
    train_df,
    x_col= 'Filepath',
    y_col= 'Type1',
    target_size= (120,120),
    color_node = 'rgba',
    class_node ='sparse',
    batch_size = 32,
    shuffle= True,
    seed= 1,
    subset = 'validation'


)
     

Found 577 validated image filenames belonging to 18 classes.
Found 144 validated image filenames belonging to 18 classes.


In [9]:
inputs = tf.keras.Input(shape=(120,120,3))

conv1= tf.keras.layers.Conv2D (filters =64, kernel_size=(8,8), activation='relu')(inputs)
pool1 = tf.keras.layers.MaxPool2D()(conv1)

conv2= tf.keras.layers.Conv2D (filters =128, kernel_size=(8,8), activation='relu')(pool1)
pool2 = tf.keras.layers.MaxPool2D()(conv2)

conv3= tf.keras.layers.Conv2D (filters =256, kernel_size=(8,8), activation='relu')(pool2)
pool3 = tf.keras.layers.MaxPool2D()(conv3)

outputs =tf.keras.layers.GlobalAveragePooling2D()(pool3)



feature_extractor = tf.keras.Model(inputs=inputs, outputs= outputs)
feature_extractor.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 120, 120, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 113, 113, 64)      12352     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 56, 56, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 49, 49, 128)       524416    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 24, 24, 128)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 17, 17, 256)       209740

In [10]:
clf_inputs = feature_extractor.input
clf_outputs = tf.keras.layers.Dense(units= 1, activation= 'sigmoid')(feature_extractor.output)

classifier = tf.keras.Model(inputs= clf_inputs, outputs= clf_outputs)


In [11]:
classifier.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 120, 120, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 113, 113, 64)      12352     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 56, 56, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 49, 49, 128)       524416    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 24, 24, 128)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 17, 17, 256)       2097

In [12]:
classifier.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)

In [13]:
history = classifier.fit(
    train_data,
    validation_data = val_data,
    batch_size = 32,
    epochs= 7,
    callbacks = [
                 tf.keras.callbacks.EarlyStopping(
                     monitor= 'val_loss',
                     patience = 5,
                     restore_best_weights = True
                 ),
                 tf.keras.callbacks.ReduceLROnPlateau()
    ]
)

c:\Users\Umut\anaconda3\lib\site-packages\PIL\Image.py:945: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch 1/7
19/19 [==============================] - 43s 2s/step - loss: 0.3174 - accuracy: 0.9337 - val_loss: 0.2497 - val_accuracy: 0.9444 - lr: 0.0010
Epoch 2/7
19/19 [==============================] - 41s 2s/step - loss: 0.2218 - accuracy: 0.9444 - val_loss: 0.2184 - val_accuracy: 0.9444 - lr: 0.0010
Epoch 3/7
19/19 [==============================] - 40s 2s/step - loss: 0.2173 - accuracy: 0.9444 - val_loss: 0.2172 - val_accuracy: 0.9444 - lr: 0.0010
Epoch 4/7
19/19 [==============================] - 43s 2s/step - loss: 0.2162 - accuracy: 0.9444 - val_loss: 0.2159 - val_accuracy: 0.9444 - lr: 0.0010
Epoch 5/7
19/19 [==============================] - 46s 2s/step - loss: 0.2155 - accuracy: 0.9444 - val_loss: 0.2155 - val_accuracy: 0.9444 - lr: 0.0010
Epoch 6/7
19/19 [==============================] - 45s 2s/step - loss: 0.2153 - accuracy: 0.9444 - val_loss: 0.2153 - val_accuracy: 0.9444 - lr: 0.0010
Epoch 7/7
19/19 [==============================] - 45s 2s/step - loss: 0.2153 - accuracy

In [14]:
feature_extractor.layers


In [15]:
feature_extractor.layers[1].weights
     

[<tf.Variable 'conv2d/kernel:0' shape=(8, 8, 3, 64) dtype=float32, numpy=
 array([[[[-1.47047732e-02,  2.85270647e-03, -3.48624550e-02, ...,
           -1.16729336e-02,  2.57532112e-02,  1.45037277e-02],
          [-4.00613435e-02, -1.10230139e-02, -3.84147838e-02, ...,
            2.25696564e-02, -1.89890601e-02,  1.64574734e-03],
          [-2.29781158e-02,  3.03232837e-02, -4.27560024e-02, ...,
           -8.88243690e-03,  2.67154984e-02, -2.71768123e-02]],
 
         [[ 3.82206775e-03,  9.96023230e-03,  1.30547229e-02, ...,
           -3.75704505e-02, -1.05551025e-02,  2.60845968e-03],
          [ 1.03656482e-02,  2.32999735e-02, -3.91716249e-02, ...,
           -1.13072270e-03,  2.70357896e-02, -2.47252099e-02],
          [-1.51023213e-02, -1.77423377e-02, -4.67167329e-03, ...,
           -5.81382401e-03, -7.46040326e-03, -2.60093771e-02]],
 
         [[-4.17472422e-02, -1.79948453e-02,  5.80177642e-03, ...,
            2.51800474e-02, -3.52219306e-03, -3.44901439e-03],
          